In [26]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [27]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [28]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [29]:
tools_list = [multiply]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [30]:
# we create a tool calling Agent by binding a list of tools to the llm

llm_with_tools = llm.bind_tools(tools_list)

In [31]:
response1 = llm_with_tools.invoke("What is 10 times 40?")

response1

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'zngz92he6', 'function': {'arguments': '{"a":10,"b":40}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 220, 'total_tokens': 239, 'completion_time': 0.070231139, 'prompt_time': 0.024965048, 'queue_time': 0.047254152, 'total_time': 0.095196187}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6567cbc4-9113-4d39-a8e5-260151e17b17-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 40}, 'id': 'zngz92he6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 19, 'total_tokens': 239})

In [32]:
response1.tool_calls

[{'name': 'multiply',
  'args': {'a': 10, 'b': 40},
  'id': 'zngz92he6',
  'type': 'tool_call'}]

In [33]:
tools_dict[response1.tool_calls[0]["name"]].invoke(response1.tool_calls[0])

ToolMessage(content='400.0', name='multiply', tool_call_id='zngz92he6')

SELF-EXERCISE

In [34]:
# Exercise: Modify the prompt in various ways
#addition tool
@tool
def addition(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def subtraction(a: float, b: float) -> float:
    """Subtract two numbers."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def division(a: float, b: float) -> float:
    """Divide two numbers."""
    return a / b

tools_list = [addition, subtraction, multiply, division]
tools_dict = {tool.name: tool for tool in tools_list}
llm_with_tools = llm.bind_tools(tools_list)

In [35]:
response1 = llm_with_tools.invoke("What is 10 times 40?")
response1.tool_calls
tools_dict[response1.tool_calls[0]["name"]].invoke(response1.tool_calls[0])

ToolMessage(content='400.0', name='multiply', tool_call_id='639yn7ejj')

In [36]:
response2 = llm_with_tools.invoke("What is the sum of 10 and 40?")
response2.tool_calls

tools_dict[response2.tool_calls[0]["name"]].invoke(response2.tool_calls[0])

ToolMessage(content='50.0', name='addition', tool_call_id='jy8mcb5v3')

In [37]:
response3 = llm_with_tools.invoke("What is the difference between 10 and 40?")

response3.tool_calls
tools_dict[response3.tool_calls[0]["name"]].invoke(response3.tool_calls[0])

ToolMessage(content='30.0', name='subtraction', tool_call_id='x8a3rrt4h')

In [38]:
response4 = llm_with_tools.invoke("What is 10 divided by 40?")

response4.tool_calls
tools_dict[response4.tool_calls[0]["name"]].invoke(response4.tool_calls[0])

ToolMessage(content='0.25', name='division', tool_call_id='m5nb0hkde')